In [17]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
import numpy as np
import re
import string

In [18]:
with open('lyrics/bob dylan_2.txt','r') as f:
    text=f.read()

text


'[Verse 1]How many roads must a man walk down before you call him a man?How many seas must the white dove sail before she sleeps in the sand?Yes, and how many times must the cannonballs fly before they\'re forever banned?[Refrain]The answer, my friend, is blowin\' in the windThe answer is blowin\' in the wind[Verse 2]Yes, and how many years can a mountain exist before it is washed to the sea?Yes, and how many years can some people exist before they\'re allowed to be free?Yes, and how many times can a man turn his head and pretend that he just doesn\'t see?[Refrain]The answer, my friend, is blowin\' in the windThe answer is blowin\' in the wind[Verse 3]Yes, and how many times must a man look up before he can see the sky?Yes, and how many ears must one man have before he can hear people cry?Yes, and how many deaths will it take till he knows that too many people have died?[Refrain]The answer, my friend, is blowin\' in the windThe answer is blowin\' in the wind\n\n[Verse 1]Come gather \'r

In [19]:
def data_clean_round1(text):
                                                    # lower case text
    text = re.sub('<[^>]*>', ' ', text)
                                    
    text = re.sub('\[.*?\]', ' ', text)                                 # remove square brackets
    # Create punctuation set without forward slash
    punct = ''.join(char for char in string.punctuation if char != '\\')
    text = re.sub('[%s]' % re.escape(punct), '', text)                  # remove punctuation except forward slash
    # Replace forward slash with /n
    
    text = re.sub(r'([a-z])([A-Z])', r'\1/n\2', text)     
    text = text.lower()                                 # convert slash to /n
    return text.strip()


In [20]:
def data_clean_round2(text):
    text = re.sub('  ', ' ', text)
    text = re.sub('   ', ' ', text)
    return text


In [21]:
data=data_clean_round1(text)
data


'how many roads must a man walk down before you call him a man/nhow many seas must the white dove sail before she sleeps in the sand/nyes and how many times must the cannonballs fly before theyre forever banned the answer my friend is blowin in the wind/nthe answer is blowin in the wind yes and how many years can a mountain exist before it is washed to the sea/nyes and how many years can some people exist before theyre allowed to be free/nyes and how many times can a man turn his head and pretend that he just doesnt see the answer my friend is blowin in the wind/nthe answer is blowin in the wind yes and how many times must a man look up before he can see the sky/nyes and how many ears must one man have before he can hear people cry/nyes and how many deaths will it take till he knows that too many people have died the answer my friend is blowin in the wind/nthe answer is blowin in the wind\n\n come gather round people wherever you roam/nand admit that the waters around you have grown/na

In [22]:
data=data_clean_round2(data)
data

'how many roads must a man walk down before you call him a man/nhow many seas must the white dove sail before she sleeps in the sand/nyes and how many times must the cannonballs fly before theyre forever banned the answer my friend is blowin in the wind/nthe answer is blowin in the wind yes and how many years can a mountain exist before it is washed to the sea/nyes and how many years can some people exist before theyre allowed to be free/nyes and how many times can a man turn his head and pretend that he just doesnt see the answer my friend is blowin in the wind/nthe answer is blowin in the wind yes and how many times must a man look up before he can see the sky/nyes and how many ears must one man have before he can hear people cry/nyes and how many deaths will it take till he knows that too many people have died the answer my friend is blowin in the wind/nthe answer is blowin in the wind\n\n come gather round people wherever you roam/nand admit that the waters around you have grown/na

In [23]:
corpus=data.split('/n')


In [24]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(corpus)

totalWords=len(tokenizer.word_index) + 1



In [25]:
input_seq = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        ngram_seq = token_list[:i+1]
        input_seq.append(ngram_seq)

In [26]:
max_seq_len = max([len(x) for x in input_seq])
input_seq = np.array(pad_sequences(input_seq, maxlen=max_seq_len, padding='pre'))

In [27]:
xs, labels = input_seq[:,:-1], input_seq[:,-1]

ys = tf.keras.utils.to_categorical(labels, num_classes=totalWords)

In [28]:
model = load_model('model.h5')

In [29]:
cp_path='weights/dyla_nweights.h5'

In [30]:
cp=tf.keras.callbacks.ModelCheckpoint(cp_path,save_freq=5,verbose=1)

In [31]:

model.load_weights(cp_path)


In [32]:
history=model.fit(xs,
                  labels,
                  epochs=20,
                  verbose=1,
                  callbacks=[cp]
                  )

Epoch 1/20
  4/452 [..............................] - ETA: 41s - loss: 18.9706 - accuracy: 0.0234
Epoch 1: saving model to weights\dyla_nweights.h5
  9/452 [..............................] - ETA: 43s - loss: 17.5532 - accuracy: 0.0243
Epoch 1: saving model to weights\dyla_nweights.h5
 14/452 [..............................] - ETA: 44s - loss: nan - accuracy: 0.0179    
Epoch 1: saving model to weights\dyla_nweights.h5
 19/452 [>.............................] - ETA: 45s - loss: nan - accuracy: 0.0132
Epoch 1: saving model to weights\dyla_nweights.h5
 24/452 [>.............................] - ETA: 44s - loss: nan - accuracy: 0.0104
Epoch 1: saving model to weights\dyla_nweights.h5
 29/452 [>.............................] - ETA: 43s - loss: nan - accuracy: 0.0086
Epoch 1: saving model to weights\dyla_nweights.h5
 34/452 [=>............................] - ETA: 42s - loss: nan - accuracy: 0.0074
Epoch 1: saving model to weights\dyla_nweights.h5
 39/452 [=>............................] - ETA

In [ ]:
seed_text=input("Give Seed")
next_words=10
for _ in range(next_words):
    token_list=tokenizer.texts_to_sequences([seed_text])[0]
    token_list=pad_sequences([token_list],maxlen=max_seq_len-1,padding='pre')
    predicted=model.predict(token_list,verbose=0)
    predicted_index = np.argmax(predicted)
    output_word=""
    for word , index in tokenizer.word_index.items():
        if index == predicted_index:
            output_word = word
            break
        
    seed_text += " " + output_word

print(seed_text)